In [ ]:
from copy import deepcopy
import math
import sys

import matplotlib.pylab as plt
import numpy as np

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

sys.path.append('../crazyflie-audio/python/')

# A. Basics

In [ ]:

def plot_spectrum(spectrum, polar, title=""):
    if polar: 
        plt.polar(BeamFormer.theta_scan, spectrum[0, :])
        for i, gt_angle_deg in enumerate(gt_angles_deg):
            plt.axvline(gt_angle_deg / 180 * np.pi, color="k")
        plt.gca().yaxis.set_visible(False)
    else:
        plt.semilogy(BeamFormer.theta_scan_deg, spectrum[0, :])
        for i, gt_angle_deg in enumerate(gt_angles_deg):
            plt.axvline(gt_angle_deg, color="k")
    plt.title(title, y=1.1)

def get_buffers(mics_list, times_list, frequency_hz, simulation_type, sanity_check=False):
    """ 
    Simulate received buffers for given mic positions and corresponding times.
    
    :param mics_list: list of mic_positions
    :param times_list: list of times corresponding to mic_positions(seconds)
    :param frequency_hz: source frequency in Hz. 
    :param simulation_type: analytical or pyroom
    
    """
    from generate_doa_results import generate_signals_analytical, generate_signals_pyroom, from_0_to_2pi
    indices = [np.argmin(np.abs(frequencies - frequency_hz))]
    
    buffer_f_list = []
    if sanity_check:
        buffer_list = []
        
    #### moving mic array ####
    for mics, time in zip(mics_list, times_list):
        buffers = np.zeros((mics.shape[0], N_BUFFER))
        
        # superposition
        for phase_offset, source in zip(phase_offsets, sources):
            if simulation_type == "pyroom":
                signals_received =  generate_signals_pyroom(
                    source, mics, frequency_hz, time, noise=signal_noise,
                    phase_offset=phase_offset) 
            elif simulation_type == "analytical":
                signals_received =  generate_signals_analytical(
                    source, mics, frequency_hz, time, noise=signal_noise, 
                    phase_offset=phase_offset) 
            else:
                raise ValueError(simulation_type)
            buffers_source = signals_received[:, time_index:time_index + N_BUFFER]
            buffers += buffers_source
            
        if sanity_check:
            buffer_list.append(buffers)

        buffer_f = np.fft.rfft(buffers)[:, indices]
        buffer_f_list.append(buffer_f.T)

    
    #### multi-mic array ####
    mics_array = np.concatenate([*mics_list])
    buffer_multimic = np.zeros((mics_array.shape[0], N_BUFFER))
    
    # superposition
    for phase_offset, source in zip(phase_offsets, sources):
        if simulation_type == "pyroom":
            signals_multimic = generate_signals_pyroom(source, mics_array, frequency_hz, time=0, noise=signal_noise, phase_offset=phase_offset)
        elif simulation_type == "analytical":
            signals_multimic = generate_signals_analytical(source, mics_array, frequency_hz, time=0, noise=signal_noise, phase_offset=phase_offset)
        else:
            raise ValueError(simulation_type)
        buffer_multimic += signals_multimic[:, time_index:time_index + N_BUFFER]
    buffer_f_multimic = np.fft.rfft(buffer_multimic)[:, indices]
    
    #### sanity check delay correction ####
    if sanity_check:
        n_plot = 50
        delta_t =  times_list[1] - times_list[0]
        print(f'period of signal {period:.2e}')
        print(f'delta_t % period: {delta_t % period :.2e}')

        times = np.arange(n_plot) / FS
        for j, (degree, buffer, time) in enumerate(zip(degrees, buffer_list, times_list_noisy)):
            fig, axs = plt.subplots(mics_drone.shape[0], 2, sharex=True)
            fig.set_size_inches(5, 5)
            fig.suptitle(f'degree {degree}')
            
            for i in range(buffer.shape[0]): # n_mics
                
                label=f"mic{i}"
                axs[i, 0].plot(times, buffer_multimic[i, :n_plot], label=label, color=f"C{i}")
                axs[i, 0].set_title(label)
                
                phase_shift_deg = from_0_to_2pi(-2 * np.pi * time * frequency_hz) * 180 / np.pi
                time_shift = period * phase_shift_deg / 360
                axs[i, 1].axvline(time_shift)
                axs[i, 1].plot(times, buffer[i, :n_plot], color=f"C{i}")
                axs[i, 1].set_title(label)
            
    return buffer_f_list, buffer_f_multimic

In [ ]:
from audio_stack.beam_former import rotate_mics
from constants import SPEED_OF_SOUND
from mic_array import get_square_array, get_uniform_array

from geometry import Context
from crazyflie_description_py.parameters import N_BUFFER, FS

dir_name = 'plots_doa/theory'

simulation_type = 'pyroom'
#simulation_type = 'analytical'

method = 'das'
#method = 'das'

plot_covariance = False

frequencies = np.fft.rfftfreq(N_BUFFER, 1/FS)

frequency_hz = frequencies[4000 * N_BUFFER // FS] # Hz
print(frequency_hz)

gt_distance = 1  # meters, distance of source
gt_angles_deg = [50] # degrees, angle of source
phase_offsets = [0.0] # phase offsets of each source

angular_velocity_deg = 20 # deg/sec, velocity of drone

max_distance = SPEED_OF_SOUND / frequency_hz / 2

context = Context.get_crazyflie_setup()
mics_drone = context.mics

#mics_drone = get_square_array(baseline=baseline, delta=0) # 4 x 2
#mics_drone = get_uniform_array(2, baseline=baseline) # 4 x 2
mics_drone -= np.mean(mics_drone, axis=0) # center the drone

# sanity check
#times_list = np.array([0, 0.8, 1.5]) #3., 6.]) #1, 1.5]) # in seconds

# uniform array, works very well
degrees = [30, 60, 90]
times_list = np.array([d/angular_velocity_deg for d in degrees]) # in seconds

In [ ]:
### parameters
time_index = 2000 # idx where signal is non-zero for all positions, found heuristically
period = 1/frequency_hz

# make sure sampling times coincide with times
times_list = np.round([(t * FS) / FS for t in times_list])

signal_noise = 1e-3  # noise added to signals
mics_noise = 0 #1e-3  # noise added to mic positions (rigid)
degree_noise = 0 #5 # noise added to each movement, in degrees
# TODO(FD) add translation imprecision
time_quantization = 6 # number of decimal places to keep
time_noise = 0 #1e-6 # noise added to recording times

In [ ]:
from plotting_tools import save_fig
from generate_doa_results import DURATION
np.random.seed(1)

assert DURATION > max(times_list)

sources = []
for gt_angle_deg in gt_angles_deg:
    gt_angle_rad = gt_angle_deg / 180 * np.pi
    sources.append(gt_distance * np.array([np.cos(gt_angle_rad), np.sin(gt_angle_rad)]))

# create noisy versions
mics_drone_noisy = deepcopy(mics_drone)
times_list_noisy = deepcopy(times_list)
degrees_noisy = deepcopy(degrees)
if mics_noise > 0:
    mics_drone_noisy += np.random.normal(scale=mics_noise, size=mics_drone.shape)
if time_noise > 0:
    times_list_noisy += np.random.normal(scale=time_noise, size=times_list.shape)
if time_quantization > 0:
    times_list_noisy = np.round(times_list_noisy, time_quantization)
if degree_noise > 0:
    degrees_noisy += np.random.normal(scale=degree_noise, size=degrees.shape)

# we think we move to "degrees" position but we actually move to degrees_noisy.
mics_list = [rotate_mics(mics_drone_noisy, orientation_deg=degree) for degree in degrees_noisy]

fig = plt.figure()
fig.set_size_inches(5, 5)
if mics_noise > 0:
    for i, (degree, mics) in enumerate(zip(degrees, mics_list)):
        plt.scatter(*mics.T, label=f'noisy {degree:.0f} deg', color=f"C{i}")
        
mics_clean = [rotate_mics(mics_drone, orientation_deg=degree) for degree in degrees]
for i, (degree, mics) in enumerate(zip(degrees, mics_clean)):
    plt.scatter(*mics.T, label=f'mics at {degree:.0f}deg', color=f"C{i}", marker="x")
plt.axis('equal')
mic_radius = np.max(np.linalg.norm(mics, axis=1)) * 2.0
plt.plot([0, mic_radius * np.cos(gt_angle_rad)], 
         [0, mic_radius * np.sin(gt_angle_rad)], color='k', label='source direction')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.title('simulation setup')
plt.legend(bbox_to_anchor=[1, 1], loc='upper left')
save_fig(fig, f'{dir_name}/noiseless-setup.pdf')
pass

In [ ]:
# sanity checks
for mics, time in zip(mics_list, times_list):
    fig, ax = plt.subplots()
    
    buffers = np.zeros((mics.shape[0], N_BUFFER))
    for gt_angle_deg in gt_angles_deg:
        gt_angle_rad = gt_angle_deg * np.pi / 180.0
        source = gt_distance * np.array([np.cos(gt_angle_rad), np.sin(gt_angle_rad)])
        signals_received_an =  generate_signals_analytical(
            source, mics, frequency_hz, time, noise=signal_noise, ax=ax) 
        signals_received_py =  generate_signals_pyroom(
            source, mics, frequency_hz, time, noise=signal_noise, ax=ax) 
        
        if simulation_type == "pyroom":
            buffers += signals_received_py[:, time_index:time_index + N_BUFFER]
        elif simulation_type == "analytical":
            buffers += signals_received_an[:, time_index:time_index + N_BUFFER]
        else:
            raise ValueError(simulation_type)
            
    ax.set_xlim(1300, 1400) 
    ax.legend()
    ax.set_title(f'signals at time {time:.3f}')
    
    fig, axs = plt.subplots(buffers.shape[0])
    n_plot = 50
    fig.suptitle(f'first {n_plot} samples of received buffers')
    for i, ax in enumerate(axs):
        ax.plot(buffers[i, :n_plot], color=f'C{i}')

In [ ]:
buffer_f_list, buffer_f_multimic = get_buffers(mics_list, times_list, frequency_hz, simulation_type, sanity_check=True)

# 3. Do beamforming, separately

In [ ]:
from audio_stack.beam_former import BeamFormer

def separate_doa(plot=False, polar=True):
    for buffer_f, mics, degree in zip(buffer_f_list, mics_list, degrees) :
        beam_former = BeamFormer(mic_positions=mics)
        R = beam_former.get_correlation(buffer_f)
        
        if method == 'das':
            spectrum = beam_former.get_das_spectrum(R, np.array([frequency_hz]))
        else:
            spectrum = beam_former.get_mvdr_spectrum(R, np.array([frequency_hz]), 
                                                     inverse='pinv')

        if plot:
            
            if plot_covariance:
                plt.figure()
                plt.matshow(np.angle(R[0]))
                plt.colorbar()
                
            fig = plt.figure()
            plot_spectrum(spectrum, polar, title=f'drone rotation {degree}deg')
            
separate_doa(plot=True)

# 3.2 Multi-mic DOA

In [ ]:
def multi_mic_doa(plot=False, polar=True, extension='.pdf'):
    mics_array = np.concatenate([*mics_list])
    beam_former = BeamFormer(mics_array)
    R_multimic = beam_former.get_correlation(buffer_f_multimic.T)
    frequencies = np.array([frequency_hz])
    
    if method == 'das':
        spectrum_multimic = beam_former.get_das_spectrum(R_multimic, frequencies)
    else:
        spectrum_multimic = beam_former.get_mvdr_spectrum(R_multimic, frequencies)

    if plot:
        if plot_covariance:
            plt.figure()
            plt.matshow(np.angle(R_multimic[0]))
            
        fig = plt.figure()
        plot_spectrum(spectrum_multimic, polar, title=f'{mics_array.shape[0]}-microphone array')
        save_fig(fig, f'{dir_name}/multimic{extension}')
        
multi_mic_doa(plot=True, extension='_one.pdf')

# 3.3 Kinetic DOA

In [ ]:
def kinetic_doa(plot=False, polar=True, extension='.pdf'):
    frequencies = np.array([frequency_hz])

    beam_former = BeamFormer(mic_positions=mics_drone)
    beam_former.init_multi_estimate(frequencies, combination_n=len(degrees))
    beam_former.init_dynamic_estimate(frequencies, combination_n=len(degrees), 
                                      combination_method='sum', normalization_method='none')

    if plot:
        fig = plt.figure()
        
    for i, (sig_f, time) in enumerate(zip(buffer_f_list, times_list_noisy)):
        beam_former.add_to_multi_estimate(sig_f, frequencies, time, degrees[i])
        spec = beam_former.add_signals_to_dynamic_estimates(sig_f, frequencies, degrees[i], method=method)
        
        if plot:
            plot_spectrum(spec, polar, title='shifted spectra')
            
    spectrum_dynamic = beam_former.get_dynamic_estimate()
    spectrum_delayed = beam_former.get_multi_estimate(method=method)

    if plot:
        save_fig(fig, f'{dir_name}/shifted{extension}')
        
        fig = plt.figure()
        plot_spectrum(spectrum_dynamic, polar, title='spectrum dynamic')
        save_fig(fig, f'{dir_name}/dynamic{extension}')
        
        
        fig = plt.figure()
        plot_spectrum(spectrum_delayed, polar, title='spectrum delayed')
        save_fig(fig, f'{dir_name}/delayed{extension}')
        
kinetic_doa(plot=True, extension='_one.pdf')

# B. Multiple source test

In [ ]:
polar = True
method = "das"

gt_angles_deg = [10, 90]#, 180, 270]
phase_offsets = np.random.uniform(0, 2*np.pi, size=len(gt_angles_deg))
sources = []
simulation_type = "analytical"

plt.figure()
plt.scatter(*mics_drone.T)
for gt_angle_deg in gt_angles_deg:
    gt_angle_rad = gt_angle_deg * np.pi / 180.0
    source = gt_distance * np.array([np.cos(gt_angle_rad), np.sin(gt_angle_rad)])
    sources.append(source)
    plt.scatter(*source)
    plt.axis('equal')

buffer_f_list, buffer_f_multimic = get_buffers(mics_list, times_list, frequency_hz, simulation_type, sanity_check=False)

In [ ]:
separate_doa(plot=True, polar=polar)

In [ ]:
multi_mic_doa(plot=True, polar=polar, extension='_two.pdf')

In [ ]:
kinetic_doa(plot=True, polar=polar, extension='_two.pdf')

# Moving noise source (like propellers)

# Full pipeline

In [ ]:
import pandas as pd

from generate_doa_results import GT_ANGLE_DEG

label_list = ['spectrum_dynamic', 'spectrum_delayed', 'spectrum_multimic']

#fname = 'results/degree_noise.pkl'; plot_by = "degree_noise"; unit="deg"
fname = 'results/time_noise.pkl'; plot_by = "time_noise"; unit="s"

df = pd.read_pickle(fname)
freq_i = 0 # only have one frequency.

n_angles = df.iloc[0].spectrum_multimic.shape[1] # n_freq x n_angles
angles = np.linspace(0, 360, n_angles)

noise_levels = df[plot_by].unique()
n_noise_levels = len(noise_levels)
print(n_noise_levels)
n_it = len(df.it.unique())
boxplot_mat = np.empty((len(label_list), n_it, n_noise_levels)) # each slice will be one boxplot

for d, (noise_level, df_noise_level) in enumerate(df.groupby(plot_by)):
    fig, axs = plt.subplots(1, len(label_list)) #sharey=True, sharex=True)
    fig.suptitle(f"{plot_by.replace('_', ' ')} {noise_level:.1g}{unit}", y=1.1)
    fig.set_size_inches(10, 3)
    for counter, (__, row) in enumerate(df_noise_level.iterrows()):
        for l, label in enumerate(label_list):
            if counter <= 3:
                
                spec = row[label][freq_i]
                spec = (spec - np.min(spec)) / (np.max(spec) - np.min(spec))
                axs[l].semilogy(angles, spec, label=f'it{counter}')
                #axs[l].set_ylim(1e-9, 2)
                
            # calculate angle estimate
            est_angle_deg = angles[np.argmax(row[label][freq_i])]
            unwrap_angles = np.unwrap(np.array([est_angle_deg, GT_ANGLE_DEG]) / 180 * np.pi)
            error_deg = abs(unwrap_angles[1] - unwrap_angles[0])*180/np.pi
            boxplot_mat[l, counter, d] = error_deg
        
    for label, ax in zip(label_list, axs):
        ax.axvline(GT_ANGLE_DEG, color='black', ls=':')
        ax.set_xlabel('angle [deg]')
        ax.set_title(label)
    axs[0].set_ylabel('spectrum [-]')

In [ ]:
fig_box, ax_box = plt.subplots(1, len(label_list), sharex=True, sharey=True)
fig_box.set_size_inches(10, 5)
for l, label in enumerate(label_list):
    ax_box[l].boxplot(boxplot_mat[l], positions=range(len(noise_levels)), widths=0.8)
    ax_box[l].set_xticklabels([f"{n:.1g}" for n in noise_levels], rotation=90)
    ax_box[l].set_title(label)
    ax_box[l].set_xlabel(f"{plot_by.replace('_', ' ')}[{unit}]")
ax_box[0].set_ylabel('absolute error [deg]')

Next simulations: 
- add translational noise
- add different noise types (drift vs. Gaussian)
- do L2 comparison between multimic and delayed spectra vs. translational / rotational noise 